# RAG 검색 시스템 구축 - AI Agent & RAG 주제

In [1]:
# API KEY를 환경변수로 관리하기 위한 설정 파일
from dotenv import load_dotenv
import os

# API KEY 정보로드
load_dotenv()

True

In [2]:
import bs4
from langchain import hub
from langchain_text_splitters import RecursiveCharacterTextSplitter, CharacterTextSplitter
from langchain_community.document_loaders import WebBaseLoader
from langchain_community.vectorstores import FAISS
from langchain_chroma import Chroma
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_core.prompts import PromptTemplate
from langchain_openai import ChatOpenAI, OpenAIEmbeddings

USER_AGENT environment variable not set, consider setting it to identify your requests.


## 1. 문서 로드 - AI Agent & RAG 기술 블로그

In [3]:
# AI Agent & RAG 관련 기술 블로그 URL
urls = [
    "https://python.langchain.com/docs/tutorials/rag/",
    "https://python.langchain.com/docs/concepts/vectorstores/",
    "https://python.langchain.com/docs/concepts/text_splitters/",
    "https://python.langchain.com/docs/concepts/embedding_models/",
    "https://blog.langchain.dev/deconstructing-rag/"
]

# Document Loader 설정
loader = WebBaseLoader(
    web_paths=urls,
    # bs_kwargs=dict(
    #     parse_only=bs4.SoupStrainer(
    #         class_=("post-content", "post-title", "post-header", "article")
    #     )
    # ),
)

# 문서 로드
docs = loader.load()

# 로드 확인
print(f"총 {len(docs)}개의 문서가 로드되었습니다.\n")
for i, doc in enumerate(docs):
    print(f"[문서 {i+1}]")
    print(f"URL: {doc.metadata.get('source', 'Unknown')}")
    print(f"내용 길이: {len(doc.page_content)} characters")
    print(f"내용 미리보기: {doc.page_content[:200]}...")
    print("-" * 80)

총 5개의 문서가 로드되었습니다.

[문서 1]
URL: https://python.langchain.com/docs/tutorials/rag/
내용 길이: 22295 characters
내용 미리보기: Build a RAG agent with LangChain - Docs by LangChainSkip to main contentDocs by LangChain home pageLangChain + LangGraphSearch...‚åòKAsk AIGitHubTry LangSmithTry LangSmithSearch...NavigationLangChainB...
--------------------------------------------------------------------------------
[문서 2]
URL: https://python.langchain.com/docs/concepts/vectorstores/
내용 길이: 3799 characters
내용 미리보기: LangChain overview - Docs by LangChainSkip to main contentDocs by LangChain home pageLangChain + LangGraphSearch...âŒ˜KAsk AIGitHubTry LangSmithTry LangSmithSearch...NavigationLangChain overviewLangCh...
--------------------------------------------------------------------------------
[문서 3]
URL: https://python.langchain.com/docs/concepts/text_splitters/
내용 길이: 3799 characters
내용 미리보기: LangChain overview - Docs by LangChainSkip to main contentDocs by LangChain home pageLangChain + LangGraphSearch

## 2. 청킹 전략 비교 (Milestone 2)

### 2-1. CharacterTextSplitter

In [4]:
# CharacterTextSplitter 적용
char_text_splitter = CharacterTextSplitter(
    separator="\n\n",
    chunk_size=1000,
    chunk_overlap=100,
    length_function=len
)

char_splits = char_text_splitter.split_documents(docs)

print(f"CharacterTextSplitter 결과:")
print(f"- 총 청크 수: {len(char_splits)}")
print(f"- 첫 번째 청크 길이: {len(char_splits[0].page_content)} characters")
print(f"- 첫 번째 청크 미리보기:\n{char_splits[0].page_content[:300]}...")

Created a chunk of size 1177, which is longer than the specified 1000
Created a chunk of size 1106, which is longer than the specified 1000
Created a chunk of size 1817, which is longer than the specified 1000
Created a chunk of size 2005, which is longer than the specified 1000
Created a chunk of size 2005, which is longer than the specified 1000
Created a chunk of size 2005, which is longer than the specified 1000
Created a chunk of size 10640, which is longer than the specified 1000


CharacterTextSplitter 결과:
- 총 청크 수: 42
- 첫 번째 청크 길이: 1177 characters
- 첫 번째 청크 미리보기:
Build a RAG agent with LangChain - Docs by LangChainSkip to main contentDocs by LangChain home pageLangChain + LangGraphSearch...‚åòKAsk AIGitHubTry LangSmithTry LangSmithSearch...NavigationLangChainBuild a RAG agent with LangChainLangChainLangGraphDeep AgentsIntegrationsLearnReferenceContributePyth...


### 2-2. RecursiveCharacterTextSplitter

In [5]:
# RecursiveCharacterTextSplitter 적용
recursive_text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000,
    chunk_overlap=100,
    length_function=len
)

recursive_splits = recursive_text_splitter.split_documents(docs)

print(f"RecursiveCharacterTextSplitter 결과:")
print(f"- 총 청크 수: {len(recursive_splits)}")
print(f"- 첫 번째 청크 길이: {len(recursive_splits[0].page_content)} characters")
print(f"- 첫 번째 청크 미리보기:\n{recursive_splits[0].page_content[:300]}...")

RecursiveCharacterTextSplitter 결과:
- 총 청크 수: 63
- 첫 번째 청크 길이: 781 characters
- 첫 번째 청크 미리보기:
Build a RAG agent with LangChain - Docs by LangChainSkip to main contentDocs by LangChain home pageLangChain + LangGraphSearch...‚åòKAsk AIGitHubTry LangSmithTry LangSmithSearch...NavigationLangChainBuild a RAG agent with LangChainLangChainLangGraphDeep AgentsIntegrationsLearnReferenceContributePyth...


### 2-3. 청킹 전략 비교 분석

In [6]:
# 청킹 전략 비교
print("=" * 80)
print("청킹 전략 비교")
print("=" * 80)
print(f"\n1. CharacterTextSplitter:")
print(f"   - 총 청크 수: {len(char_splits)}")
print(f"   - 평균 청크 길이: {sum(len(chunk.page_content) for chunk in char_splits) / len(char_splits):.0f} characters")
print(f"   - 특징: 단순 구분자(\\n\\n) 기반 분할")

print(f"\n2. RecursiveCharacterTextSplitter:")
print(f"   - 총 청크 수: {len(recursive_splits)}")
print(f"   - 평균 청크 길이: {sum(len(chunk.page_content) for chunk in recursive_splits) / len(recursive_splits):.0f} characters")
print(f"   - 특징: 계층적 구분자 사용, 문맥 보존 우수")

print("\n💡 추천: RecursiveCharacterTextSplitter")
print("   - 더 일관된 청크 크기")
print("   - 문맥을 더 잘 보존")
print("   - 코드 블록과 구조를 잘 유지")

# 최종 선택
splits = recursive_splits
print(f"\n✅ 최종 선택: RecursiveCharacterTextSplitter ({len(splits)} chunks)")

청킹 전략 비교

1. CharacterTextSplitter:
   - 총 청크 수: 42
   - 평균 청크 길이: 1081 characters
   - 특징: 단순 구분자(\n\n) 기반 분할

2. RecursiveCharacterTextSplitter:
   - 총 청크 수: 63
   - 평균 청크 길이: 745 characters
   - 특징: 계층적 구분자 사용, 문맥 보존 우수

💡 추천: RecursiveCharacterTextSplitter
   - 더 일관된 청크 크기
   - 문맥을 더 잘 보존
   - 코드 블록과 구조를 잘 유지

✅ 최종 선택: RecursiveCharacterTextSplitter (63 chunks)


## 3. 임베딩 및 벡터 저장 (Milestone 3)

### 3-1. FAISS 벡터스토어

In [7]:
# OpenAI 임베딩 모델 사용
embeddings = OpenAIEmbeddings()

# FAISS 벡터스토어 생성
vectorstore_faiss = FAISS.from_documents(
    documents=splits,
    embedding=embeddings
)

print("✅ FAISS 벡터스토어 생성 완료")
print(f"   - 저장된 문서 수: {len(splits)}")
print(f"   - 임베딩 모델: OpenAI text-embedding-ada-002")

✅ FAISS 벡터스토어 생성 완료
   - 저장된 문서 수: 63
   - 임베딩 모델: OpenAI text-embedding-ada-002


In [8]:
import pandas as pd
import os
from pathlib import Path

# 홈 디렉토리 사용
cdir = '/Users/joyoungha/Desktop/project/rag-practice'
output_dir = os.path.join(cdir, 'rag_output')

# 디렉토리 생성
os.makedirs(output_dir, exist_ok=True)

# 청크 데이터
chunks_data = [{
    'chunk_id': i,
    'content': chunk.page_content,
    'content_length': len(chunk.page_content),
    'source': chunk.metadata.get('source', 'Unknown')
} for i, chunk in enumerate(splits)]

df = pd.DataFrame(chunks_data)

# CSV 저장
csv_path = os.path.join(output_dir, 'chunks.csv')
df.to_csv(csv_path, index=False, encoding='utf-8-sig')

print(f"✅ CSV 저장 완료: {csv_path}")
print(f"\n터미널에서 확인:")
print(f"cd {output_dir}")
print(f"ls -lh chunks.csv")

✅ CSV 저장 완료: /Users/joyoungha/Desktop/project/rag-practice/rag_output/chunks.csv

터미널에서 확인:
cd /Users/joyoungha/Desktop/project/rag-practice/rag_output
ls -lh chunks.csv


### 3-2. Chroma 벡터스토어 (대안)

In [9]:
# Chroma 벡터스토어 생성 (선택사항)
vectorstore_chroma = Chroma.from_documents(
    documents=splits,
    embedding=embeddings
)

print("✅ Chroma 벡터스토어 생성 완료")
print(f"   - 저장된 문서 수: {len(splits)}")
print(f"   - 임베딩 모델: OpenAI text-embedding-ada-002")

✅ Chroma 벡터스토어 생성 완료
   - 저장된 문서 수: 63
   - 임베딩 모델: OpenAI text-embedding-ada-002


### 3-3. 벡터스토어 선택

In [10]:
# 최종 사용할 벡터스토어 선택 (FAISS 사용)
vectorstore = vectorstore_chroma
retriever = vectorstore.as_retriever(
    search_kwargs={"k": 4}  # 상위 4개 결과 반환
)

print("✅ Retriever 설정 완료")
print(f"   - 벡터스토어: Chroma")
print(f"   - 검색 결과 수: 4")

✅ Retriever 설정 완료
   - 벡터스토어: Chroma
   - 검색 결과 수: 4


## 4. 검색 시스템 구현 및 테스트 (Milestone 4)

### 4-1. RAG 프롬프트 설정

In [11]:
# RAG 프롬프트 템플릿
# 1. system prompt 설정
# 2. question placeholder 설정
# 3. vector DB 검색 결과 context placeholder 설정
prompt = PromptTemplate.from_template(
    """당신은 주어진 기술 문서에 기반해서만 답변하는 AI 어시스턴트입니다.

답변 시 주의사항:
1. 주어진 문맥에서 찾을 수 있는 정보만 사용하세요.
2. 문서에 없는 내용은 ‘문서에 정보가 없습니다’라고 답하세요.
3. 기술 용어는 원문 그대로 사용하세요 (예: RAG, Vector Store, Embedding).
4. 가능하면 구체적인 예시나 코드를 포함하세요.

#Question: 
{question} 

#Context: 
{context} 

#Answer:"""
)

print("✅ RAG 프롬프트 설정 완료")

✅ RAG 프롬프트 설정 완료


### 4-2. RAG Chain 구성

In [12]:
# LLM 설정
llm = ChatOpenAI(model_name="gpt-4o-mini", temperature=0.3)

# RAG 체인 생성 (LCEL 방식)
rag_chain = (
    {"context": retriever, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

print("✅ RAG Chain 구성 완료")
print(f"   - LLM: {llm.model_name}")
print(f"   - Temperature: {llm.temperature}")

✅ RAG Chain 구성 완료
   - LLM: gpt-4o-mini
   - Temperature: 0.3


### 4-3. 테스트 질문 (최소 3개)

In [15]:
# 테스트 질문 1: RAG 기본 개념
print("="*80)
print("질문 1: RAG란 무엇인가요?")
print("="*80)

answer1 = rag_chain.invoke("RAG란 무엇인가요? 어떻게 작동하나요?")
print(answer1)
print("\n")

질문 1: RAG란 무엇인가요?
RAG( Retrieval-Augmented Generation)란 정보 검색(data retrieval)과 생성(generation) 모델을 결합한 접근 방식입니다. RAG는 사용자의 질문에 대한 답변을 생성하기 위해 외부 데이터 소스에서 정보를 검색하고, 이 검색된 정보를 LLM(대형 언어 모델)에게 프롬프트의 일부로 전달하는 과정을 포함합니다.

RAG의 작동 방식은 다음과 같습니다:

1. **사용자 질문 입력**: 사용자가 질문을 입력합니다.
2. **정보 검색**: 입력된 질문을 바탕으로 관련 문서를 검색합니다. 이 과정에서 쿼리 변환(query transformations) 기법을 사용하여 사용자 입력을 수정하여 검색의 정확성을 높일 수 있습니다.
3. **LLM에 정보 전달**: 검색된 문서와 초기 질문을 LLM에 전달하여 답변을 생성합니다.
4. **답변 반환**: LLM이 생성한 답변을 사용자에게 반환합니다.

예를 들어, LangChain을 사용하여 RAG 에이전트를 구축할 수 있습니다. 다음은 간단한 RAG 에이전트를 구현하는 코드의 예시입니다:

```python
from langchain.tools import tool

@tool
def search_documents(query):
    # 이 함수는 주어진 쿼리에 대해 문서를 검색하는 로직을 포함합니다.
    pass

def rag_agent(user_question):
    # 문서 검색
    retrieved_docs = search_documents(user_question)
    
    # LLM에 질문과 검색된 문서 전달
    answer = llm.generate_answer(user_question, retrieved_docs)
    
    return answer
```

이와 같이 RAG는 정보 검색과 생성 모델을 결합하여 보다 정확하고 유용한 답변을 제공하는 데 도움을 줍니다.




In [38]:
# 유사도 점수 포함
results = vectorstore.similarity_search_with_score("RAG란 무엇인가요?", k=4)

for i, (doc, score) in enumerate(results):
    print(f"\n[{i+1}위] 유사도: {score:.4f}")
    print(f"출처: {doc.metadata.get('source', 'Unknown')}")
    print(f"내용: {doc.page_content[:100]}...")


[1위] 유사도: 0.4697
출처: https://blog.langchain.dev/deconstructing-rag/
내용: information from a data source (or sources), and a process of passing the retrieved information dire...

[2위] 유사도: 0.4725
출처: https://blog.langchain.dev/deconstructing-rag/
내용: Deconstructing RAG



















































Skip to content














...

[3위] 유사도: 0.4984
출처: https://python.langchain.com/docs/tutorials/rag/
내용: We can see the full sequence of steps, along with latency and other metadata, in the LangSmith trace...

[4위] 유사도: 0.4989
출처: https://python.langchain.com/docs/tutorials/rag/
내용: Now let‚Äôs write the actual application logic. We want to create a simple application that takes a ...


In [13]:
test_questions = {
    # ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
    # 카테고리 1: 문서에 답이 명확히 있는 질문 (5개)
    # ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
    
    "1. 정의 (단답형)": {
        "question": "RAG란 무엇인가요?",
        "category": "문서 내 존재",
        "type": "단답형 - 정의",
        "expected": "RAG의 정의와 기본 개념 설명"
    },
    
    "2. 기능 설명 (상세형)": {
        "question": "Vector Store는 RAG 시스템에서 어떤 역할을 하나요?",
        "category": "문서 내 존재",
        "type": "상세형 - 기능 설명",
        "expected": "Vector Store의 역할과 작동 방식"
    },
    
    "3. 비교형": {
        "question": "CharacterTextSplitter와 RecursiveCharacterTextSplitter의 차이점을 비교해서 설명해주세요.",
        "category": "문서 내 존재",
        "type": "비교형",
        "expected": "두 Text Splitter의 차이점 설명"
    },
    
    "4. 파라미터 설명": {
        "question": "Text Splitter에서 chunk_size와 chunk_overlap 파라미터는 어떤 의미인가요?",
        "category": "문서 내 존재",
        "type": "단답형 - 파라미터",
        "expected": "청크 크기와 겹침의 의미 설명"
    },
    
    "5. 응용형 (실제 예시)": {
        "question": "문서에 나온 Embedding 모델을 실제로 어떻게 사용하는지 코드 예시와 함께 설명해주세요.",
        "category": "문서 내 존재",
        "type": "응용형 - 코드 예시",
        "expected": "Embedding 모델 사용법과 코드"
    },
    
    # ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
    # 카테고리 2: 문서에 없거나 애매한 질문 (3개) - Hallucination 테스트
    # ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
    
    "6. 가격 정보 (문서 외)": {
        "question": "LangChain의 가격은 얼마인가요?",
        "category": "문서 외 정보",
        "type": "단답형 - 문서 없음",
        "expected": "'문서에 해당 정보가 없습니다' 응답"
    },
    
    "7. 최신 트렌드 (시간적 범위 외)": {
        "question": "2025년 RAG 시스템의 최신 트렌드는 무엇인가요?",
        "category": "문서 외 정보",
        "type": "상세형 - 시간 범위 외",
        "expected": "'문서에 해당 정보가 없습니다' 응답"
    },
    
    "8. 특정 회사 사례 (문서 외)": {
        "question": "한국 기업 중 RAG를 가장 잘 사용하는 회사는 어디인가요?",
        "category": "문서 외 정보",
        "type": "단답형 - 구체적 사례",
        "expected": "'문서에 해당 정보가 없습니다' 응답"
    },
    
    # ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
    # 보너스: 추가 질문 (난이도 다양화)
    # ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
    
    "9. 장단점 분석": {
        "question": "RAG 시스템의 장점과 단점을 각각 설명해주세요.",
        "category": "문서 내 존재",
        "type": "분석형 - 장단점",
        "expected": "RAG의 장단점 비교"
    },
    
    "10. 프로세스 설명": {
        "question": "RAG 시스템에서 사용자 질문이 입력되면 답변이 생성되기까지 어떤 과정을 거치나요?",
        "category": "문서 내 존재",
        "type": "프로세스 설명",
        "expected": "RAG 파이프라인 단계별 설명"
    }
}

In [29]:
# 인터랙티브 셀 
question_count = 0
while True:
    # 질문 입력
    question = input("\n💬 질문을 입력하세요 (종료: exit): ")
    
    # 종료 조건
    if question.lower() in ["exit", "quit", "종료"]:
        print("\n👋 RAG Q&A 챗봇을 종료합니다.")
        break
    
    # 빈 입력 처리
    if not question.strip():
        print("⚠️  질문을 입력해주세요.")
        continue
    
    # 질문 카운트
    question_count += 1
    
    print(f"\n{'='*80}")
    print(f"질문 #{question_count}: {question}")
    print(f"{'='*80}")
    print("\n🤖 답변 생성 중...\n")
    
    try:
        # RetrievalQA 체인 실행
        answer = rag_chain.invoke(question)
        
        print("━" * 80)
        print(answer)
        print("━" * 80)
        
    except Exception as e:
        print(f"❌ 오류 발생: {e}")
        print("다시 시도해주세요.")


질문 #1: RAG 시스템의 장점과 단점을 각각 설명해주세요.

🤖 답변 생성 중...

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
RAG 시스템의 장점과 단점은 다음과 같습니다.

### 장점:
1. **정보 검색의 효율성**: RAG 시스템은 다양한 데이터 소스에서 정보를 검색하고, 이를 LLM에 직접 전달하여 보다 정확한 답변을 생성할 수 있습니다.
2. **사용자 쿼리의 변동성에 대한 강건성**: Query Transformations을 통해 사용자 입력을 수정하여 검색의 정확성을 높일 수 있습니다. 이는 사용자 질문이 잘못 표현되었을 때도 유용합니다.

### 단점:
1. **복잡성**: RAG 방법론의 다양성으로 인해 사용자들이 어디서 시작해야 할지 혼란스러울 수 있습니다. 이는 시스템의 구현과 이해를 어렵게 만들 수 있습니다.
2. **지연 시간**: RAG 체인을 사용할 경우, 여러 단계의 검색과 처리 과정이 필요하므로 응답 시간이 길어질 수 있습니다.

이러한 장점과 단점은 RAG 시스템을 설계하고 구현할 때 고려해야 할 중요한 요소들입니다.
━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

질문 #2: RAG는 얼마나 비싼가요?

🤖 답변 생성 중...

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
문서에 정보가 없습니다.
━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

질문 #3: 2025년 RAG 시스템의 최신 트렌드는 무엇인가요?

🤖 답변 생성 중...

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [17]:
print("\n" + "="*80)
print("1.11 8개 테스트 질문 자동 실행")
print("="*80)

print("""
자동으로 8개 질문을 순차 실행합니다.
각 질문에 대한 답변을 확인하세요.

특히 주목할 점:
  ✓ 문서 내 질문 (1-5번): 정확한 답변 생성 여부
  ✓ 문서 외 질문 (6-8번): "문서에 정보가 없습니다" 응답 여부 (Hallucination 방지)
""")

# 결과 저장용
test_results = []

# 각 질문 실행
for idx, (key, data) in enumerate(test_questions.items(), 1):
    print("\n" + "=" * 80)
    print(f"테스트 {idx}/{len(test_questions)}: {key}")
    print("=" * 80)
    print(f"질문: {data['question']}")
    print(f"유형: {data['type']}")
    print(f"기대 결과: {data['expected']}")
    print("\n🤖 답변:")
    print("━" * 80)
    
    try:
        # RetrievalQA 체인 실행
        answer = rag_chain.invoke(data['question'])
        print(answer)
        
        # 결과 저장
        test_results.append({
            'number': idx,
            'question': data['question'],
            'category': data['category'],
            'type': data['type'],
            'answer': answer,
            'success': True
        })
        
    except Exception as e:
        print(f"❌ 오류 발생: {e}")
        test_results.append({
            'number': idx,
            'question': data['question'],
            'category': data['category'],
            'type': data['type'],
            'answer': f"Error: {e}",
            'success': False
        })
    
    print("━" * 80)

print("\n" + "=" * 80)
print("✅ 전체 테스트 완료")
print("=" * 80)
print(f"총 {len(test_results)}개 질문 실행 완료")
print(f"성공: {sum(1 for r in test_results if r['success'])}개")
print(f"실패: {sum(1 for r in test_results if not r['success'])}개")

print("\n" + "="*80)
print("1.12 테스트 결과 분석")
print("="*80)

import pandas as pd

# DataFrame으로 결과 정리
df_results = pd.DataFrame([
    {
        '번호': r['number'],
        '질문': r['question'][:50] + '...' if len(r['question']) > 50 else r['question'],
        '카테고리': r['category'],
        '유형': r['type'],
        '답변 길이': len(r['answer']),
        '성공': '✅' if r['success'] else '❌'
    }
    for r in test_results
])

print("\n[테스트 결과 요약표]")
print(df_results.to_string(index=False))

# 카테고리별 분석
print("\n" + "━" * 80)
print("[카테고리별 분석]")
print("━" * 80)

category_summary = df_results.groupby('카테고리').size()
for category, count in category_summary.items():
    print(f"  {category}: {count}개 질문")

# 유형별 분석
print("\n" + "━" * 80)
print("[질문 유형 분석]")
print("━" * 80)

type_summary = df_results.groupby('유형').size()
for qtype, count in type_summary.items():
    print(f"  {qtype}: {count}개")

# Hallucination 테스트 결과
print("\n" + "━" * 80)
print("[Hallucination 방지 테스트 (문서 외 질문)]")
print("━" * 80)

doc_outside = [r for r in test_results if r['category'] == '문서 외 정보']
for r in doc_outside:
    print(f"\n질문 {r['number']}: {r['question']}")
    
    # "문서에 없다"는 표현이 포함되어 있는지 확인
    keywords = ['문서에', '정보가 없', '찾을 수 없', '제공되지 않', '없습니다']
    has_disclaimer = any(keyword in r['answer'] for keyword in keywords)
    
    if has_disclaimer:
        print("  ✅ 올바른 응답: '문서에 정보 없음' 명시")
    else:
        print("  ⚠️  주의: Hallucination 가능성 있음")
    
    print(f"  답변 미리보기: {r['answer'][:100]}...")

print("\n" + "=" * 80)


1.11 8개 테스트 질문 자동 실행

자동으로 8개 질문을 순차 실행합니다.
각 질문에 대한 답변을 확인하세요.

특히 주목할 점:
  ✓ 문서 내 질문 (1-5번): 정확한 답변 생성 여부
  ✓ 문서 외 질문 (6-8번): "문서에 정보가 없습니다" 응답 여부 (Hallucination 방지)


테스트 1/10: 1. 정의 (단답형)
질문: RAG란 무엇인가요?
유형: 단답형 - 정의
기대 결과: RAG의 정의와 기본 개념 설명

🤖 답변:
━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
RAG는 "Retrieval-Augmented Generation"의 약자로, 데이터 소스에서 정보를 검색하고, 검색된 정보를 LLM(대형 언어 모델)에 프롬프트의 일부로 직접 전달하는 과정을 포함합니다. RAG 방법론은 최근 몇 달 동안 크게 확장되어 사용자들이 다양한 접근 방식을 이해하는 데 혼란을 겪고 있습니다. RAG의 주요 테마 중 하나는 쿼리 변환으로, 이는 사용자 입력의 변동성에 강한 검색을 가능하게 하기 위한 접근 방식입니다.

예를 들어, 사용자가 "누가 우승했는가?"라는 질문을 했을 때, RAG는 이 질문을 더 잘 이해하고 검색할 수 있도록 쿼리를 변환하는 방법을 사용할 수 있습니다. RAG 에이전트는 사용자의 질문을 받아 관련 문서를 검색하고, 검색된 문서와 초기 질문을 모델에 전달하여 답변을 반환하는 간단한 애플리케이션 로직을 구현할 수 있습니다. 

구체적인 예로, LangChain을 사용하여 RAG 에이전트를 구현할 수 있습니다. 다음은 RAG 에이전트를 구성하는 간단한 코드 예시입니다:

```python
from langchain.tools import tool

# RAG 에이전트의 도구를 구현
@tool
def search_tool(query):
    # 벡터 저장소에서 쿼리에 대한 문서를 검색하는 로직
    pass

# RAG 에이전

## Rewrite Query

In [18]:
print("="*80)
print("2.1 재작성 프롬프트 설계")
print("="*80)

# 재작성 프롬프트
rewrite_prompt_template = """당신은 검색 쿼리 최적화 전문가입니다.

주어진 질문을 분석하고, 더 명확하고 검색하기 좋게 재작성하세요.

재작성 규칙:
1. 애매한 표현은 구체적으로 변환
2. 약어는 전체 용어로 확장
3. 맥락이 부족하면 추가
4. 검색 키워드를 명확히 표현
5. 한 문장으로 작성

예시:
- "RAG" → "RAG (Retrieval-Augmented Generation)의 정의와 작동 원리"
- "벡터 스토어" → "Vector Store의 역할과 RAG 시스템에서의 사용 방법"
- "좋은 방법?" → "효과적인 문서 검색 방법과 최적화 전략"

원본 질문: {question}

재작성된 질문 (한 문장으로):"""

rewrite_prompt = PromptTemplate.from_template(rewrite_prompt_template)

print("✅ 재작성 프롬프트 생성 완료")
print("\n[프롬프트 핵심 요소]")
print("  1. 역할: 검색 쿼리 최적화 전문가")
print("  2. 규칙: 5가지 재작성 원칙")
print("  3. 예시: 3가지 변환 패턴")
print("  4. 제약: 한 문장으로 출력")

2.1 재작성 프롬프트 설계
✅ 재작성 프롬프트 생성 완료

[프롬프트 핵심 요소]
  1. 역할: 검색 쿼리 최적화 전문가
  2. 규칙: 5가지 재작성 원칙
  3. 예시: 3가지 변환 패턴
  4. 제약: 한 문장으로 출력


In [19]:
print("\n" + "="*80)
print("2.1 재작성 RetrievalQA 체인 구현")
print("="*80)

# Step 1: 재작성 체인
rewrite_chain = rewrite_prompt | llm | StrOutputParser()

# Step 2: 재작성 + RAG 통합 체인
rewrite_rag_chain = (
    # 먼저 질문을 재작성
    RunnablePassthrough.assign(
        rewritten_question=lambda x: rewrite_chain.invoke({"question": x["question"]})
    )
    # 재작성된 질문으로 검색
    | RunnablePassthrough.assign(
        context=lambda x: retriever.invoke(x["rewritten_question"])
    )
    # 프롬프트에 원본 질문 + 재작성된 질문 + 문맥 전달
    | PromptTemplate.from_template(
        """당신은 주어진 기술 문서에 기반해서만 답변하는 AI 어시스턴트입니다.

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
제공된 문서:
━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
{context}

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
원본 질문: {question}
재작성된 질문: {rewritten_question}
━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

답변 (1줄 요약 + 상세 설명 + 근거 문서):"""
    )
    | llm
    | StrOutputParser()
)

print("✅ 재작성 RetrievalQA 체인 구성 완료")
print("\n[체인 구조]")
print("""
    원본 질문
        ↓
    rewrite_chain (질문 재작성)
        ↓
    재작성된 질문
        ↓
    retriever (재작성된 질문으로 검색)
        ↓
    문서 검색 결과
        ↓
    prompt (원본 + 재작성 + 문맥)
        ↓
    llm (답변 생성)
        ↓
    최종 답변
""")


2.1 재작성 RetrievalQA 체인 구현
✅ 재작성 RetrievalQA 체인 구성 완료

[체인 구조]

    원본 질문
        ↓
    rewrite_chain (질문 재작성)
        ↓
    재작성된 질문
        ↓
    retriever (재작성된 질문으로 검색)
        ↓
    문서 검색 결과
        ↓
    prompt (원본 + 재작성 + 문맥)
        ↓
    llm (답변 생성)
        ↓
    최종 답변



In [ ]:
print("\n" + "="*80)
print("2.1 기본 RAG vs 재작성 RAG 답변 비교")
print("="*80)

# 테스트 질문
comparison_question = "chunk overlap"

print(f"질문: '{comparison_question}'\n")

print("━" * 80)
print("🔹 기본 RAG 답변")
print("━" * 80)
basic_answer = rag_chain.invoke(comparison_question)
print(basic_answer)

print("\n" + "━" * 80)
print("🔸 재작성 RAG 답변")
print("━" * 80)
rewrite_answer = rewrite_rag_chain.invoke({"question": comparison_question})
print(rewrite_answer)

print("\n" + "=" * 80)
print("💡 비교 분석")
print("=" * 80)
print("""
기본 RAG:
  • 짧은 질문 "RAG"를 그대로 검색
  • 약어만으로는 맥락 부족 가능

재작성 RAG:
  • "RAG"를 "RAG (Retrieval-Augmented Generation)의 정의와 작동 원리"로 확장
  • 더 명확한 검색어로 관련 문서 정확히 찾음
  • 답변 품질 향상
""")


2.1 기본 RAG vs 재작성 RAG 답변 비교
질문: 'chunk overlap'

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
🔹 기본 RAG 답변
━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
문서에서 "chunk overlap"에 대한 정보는 없습니다.

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
🔸 재작성 RAG 답변
━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
**요약:** Chunk overlap은 데이터 처리에서 정보의 중복을 줄이고, 더 나은 검색 결과를 위해 문서를 나누는 방법으로 중요하다.

**상세 설명:** Chunk overlap은 문서를 여러 개의 작은 청크로 나눌 때, 각 청크가 서로 겹치는 부분을 가지도록 하는 기법이다. 이는 특히 문서의 의미가 청크의 경계에서 손실되지 않도록 도와주며, 검색 시 관련 정보를 더 잘 포착할 수 있게 한다. 예를 들어, 문서의 중요한 내용이 청크의 경계에 위치할 경우, 겹침이 없으면 해당 정보가 누락될 수 있다. 따라서, chunk overlap을 활용하면 LLM(대형 언어 모델)이 더 풍부한 문맥을 이해하고, 더 정확한 응답을 생성할 수 있다.

**근거 문서:** 제공된 문서에서는 chunk size와 관련하여, 문서의 분할 방식이 정보 로딩에 미치는 영향을 강조하고 있으며, 이는 chunk overlap의 중요성을 암시한다. "chunk size controls how much information we load into the context window"라는 문구에서 이를 확인할 수 있다.

💡

## Multi Query

In [22]:
print("\n" + "="*80)
print("2.2 다중 쿼리 생성 프롬프트 설계")
print("="*80)

# 다중 쿼리 생성 프롬프트
multi_query_prompt_template = """당신은 검색 쿼리 최적화 전문가입니다.

주어진 질문을 3가지 다른 관점으로 재구성하세요.
각 질문은 같은 정보를 찾되, 다른 각도에서 접근해야 합니다.

재구성 전략:
1. 다른 용어 사용 (동의어, 관련 개념)
2. 다른 관점 적용 (기능, 장점, 사용법 등)
3. 다른 세부 수준 (추상적 ↔ 구체적)

원본 질문: {question}

3가지 재구성된 질문 (각각 한 줄로):
1."""

multi_query_prompt = PromptTemplate.from_template(multi_query_prompt_template)

print("✅ 다중 쿼리 생성 프롬프트 완료")
print("\n[프롬프트 핵심 요소]")
print("  1. 역할: 검색 쿼리 최적화 전문가")
print("  2. 목표: 3가지 다른 관점의 질문 생성")
print("  3. 전략: 용어/관점/세부수준 변경")


2.2 다중 쿼리 생성 프롬프트 설계
✅ 다중 쿼리 생성 프롬프트 완료

[프롬프트 핵심 요소]
  1. 역할: 검색 쿼리 최적화 전문가
  2. 목표: 3가지 다른 관점의 질문 생성
  3. 전략: 용어/관점/세부수준 변경


In [24]:
print("\n" + "="*80)
print("2.2 다중 쿼리 생성 체인 구현")
print("="*80)

# 다중 쿼리 생성 체인
multi_query_generation_chain = multi_query_prompt | llm | StrOutputParser()

# 쿼리 파싱 함수 (생성된 텍스트를 리스트로 변환)
def parse_queries(generated_text: str):
    """
    LLM이 생성한 다중 쿼리 텍스트를 파싱하여 리스트로 반환
    
    입력 예시:
    "1. 첫 번째 질문
     2. 두 번째 질문
     3. 세 번째 질문"
    
    출력 예시:
    ["첫 번째 질문", "두 번째 질문", "세 번째 질문"]
    """
    lines = generated_text.strip().split('\n')
    queries = []
    
    for line in lines:
        # "1.", "2.", "3." 등의 번호 제거
        line = line.strip()
        if line and (line[0].isdigit() or line.startswith('-')):
            # 번호나 불릿 제거
            query = line.split('.', 1)[-1].strip()
            query = query.lstrip('- ').strip()
            if query:
                queries.append(query)
    
    return queries

print("✅ 다중 쿼리 생성 체인 완료")
print("✅ 쿼리 파싱 함수 정의 완료")


2.2 다중 쿼리 생성 체인 구현
✅ 다중 쿼리 생성 체인 완료
✅ 쿼리 파싱 함수 정의 완료


In [25]:
print("\n" + "="*80)
print("2.2 다중 쿼리 생성 테스트")
print("="*80)

# 테스트 질문
test_question = "Vector Store의 역할은 무엇인가요?"

print(f"원본 질문: {test_question}\n")
print("다중 쿼리 생성 중...\n")
print("━" * 80)

# 다중 쿼리 생성
generated_text = multi_query_generation_chain.invoke({"question": test_question})
print(generated_text)
print("━" * 80)

# 쿼리 파싱
queries = parse_queries(generated_text)

print(f"\n✅ 생성된 쿼리 수: {len(queries)}개\n")
print("[파싱된 쿼리 리스트]")
for i, q in enumerate(queries, 1):
    print(f"  {i}. {q}")

# 원본 질문도 포함
all_queries = [test_question] + queries
print(f"\n[검색에 사용할 전체 쿼리] (원본 포함)")
for i, q in enumerate(all_queries, 1):
    print(f"  {i}. {q}")


2.2 다중 쿼리 생성 테스트
원본 질문: Vector Store의 역할은 무엇인가요?

다중 쿼리 생성 중...

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
1. 벡터 저장소의 기능은 무엇인가요?
2. 벡터 저장소를 활용하는 주요 이점은 무엇인가요?
3. 벡터 저장소가 데이터 검색 및 관리에서 구체적으로 어떻게 작동하나요?
━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

✅ 생성된 쿼리 수: 3개

[파싱된 쿼리 리스트]
  1. 벡터 저장소의 기능은 무엇인가요?
  2. 벡터 저장소를 활용하는 주요 이점은 무엇인가요?
  3. 벡터 저장소가 데이터 검색 및 관리에서 구체적으로 어떻게 작동하나요?

[검색에 사용할 전체 쿼리] (원본 포함)
  1. Vector Store의 역할은 무엇인가요?
  2. 벡터 저장소의 기능은 무엇인가요?
  3. 벡터 저장소를 활용하는 주요 이점은 무엇인가요?
  4. 벡터 저장소가 데이터 검색 및 관리에서 구체적으로 어떻게 작동하나요?


In [26]:
print("\n" + "="*80)
print("2.2 각 쿼리로 독립 검색 및 결과 통합")
print("="*80)

# 각 쿼리로 검색
all_search_results = []

print("\n[각 쿼리별 검색 결과]\n")

for i, query in enumerate(all_queries, 1):
    print(f"쿼리 {i}: {query}")
    
    # 검색 수행
    docs = retriever.invoke(query)
    all_search_results.append(docs)
    
    print(f"  → 검색 결과: {len(docs)}개 문서")
    print(f"     첫 문서: {docs[0].page_content[:60]}...")
    print()

# 결과 통합 (중복 제거)
print("━" * 80)
print("[검색 결과 통합]")
print("━" * 80)

# 문서 내용을 기준으로 중복 제거
seen_contents = set()
unique_docs = []

for docs in all_search_results:
    for doc in docs:
        # 문서 내용의 앞 100자로 중복 체크
        doc_signature = doc.page_content[:100]
        if doc_signature not in seen_contents:
            seen_contents.add(doc_signature)
            unique_docs.append(doc)

print(f"전체 검색 문서: {sum(len(docs) for docs in all_search_results)}개")
print(f"중복 제거 후: {len(unique_docs)}개")
print(f"\n💡 다중 쿼리를 통해 {len(unique_docs)}개의 다양한 문서 수집")


2.2 각 쿼리로 독립 검색 및 결과 통합

[각 쿼리별 검색 결과]

쿼리 1: Vector Store의 역할은 무엇인가요?
  → 검색 결과: 4개 문서
     첫 문서: Integrations: 30+ integrations to choose from.
Interface: AP...

쿼리 2: 벡터 저장소의 기능은 무엇인가요?
  → 검색 결과: 4개 문서
     첫 문서: at this task, enabling data privacy (see our templates here ...

쿼리 3: 벡터 저장소를 활용하는 주요 이점은 무엇인가요?
  → 검색 결과: 4개 문서
     첫 문서: at this task, enabling data privacy (see our templates here ...

쿼리 4: 벡터 저장소가 데이터 검색 및 관리에서 구체적으로 어떻게 작동하나요?
  → 검색 결과: 4개 문서
     첫 문서: at this task, enabling data privacy (see our templates here ...

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
[검색 결과 통합]
━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
전체 검색 문서: 16개
중복 제거 후: 5개

💡 다중 쿼리를 통해 5개의 다양한 문서 수집


In [ ]:
print("\n" + "="*80)
print("2.2 다중 쿼리 RetrievalQA 체인 구현")
print("="*80)

def multi_query_retrieval(question: str):
    """
    다중 쿼리를 생성하고 각각 검색한 후 결과를 통합
    """
    # 1. 다중 쿼리 생성
    generated_text = multi_query_generation_chain.invoke({"question": question})
    queries = parse_queries(generated_text)
    
    # 2. 원본 질문도 포함
    all_queries = [question] + queries
    
    # 3. 각 쿼리로 검색
    all_results = []
    for q in all_queries:
        docs = retriever.invoke(q)
        all_results.extend(docs)
    
    # 4. 중복 제거
    seen = set()
    unique_docs = []
    for doc in all_results:
        sig = doc.page_content[:100]
        if sig not in seen:
            seen.add(sig)
            unique_docs.append(doc)
    
    return unique_docs

# 다중 쿼리 RetrievalQA 체인
multi_query_rag_chain = (
    RunnablePassthrough.assign(
        context=lambda x: multi_query_retrieval(x["question"])
    )
    | PromptTemplate.from_template(
        """당신은 주어진 기술 문서에 기반해서만 답변하는 AI 어시스턴트입니다.

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
제공된 문서 (다중 쿼리로 검색):
━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
{context}

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
질문: {question}
━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

답변 (1줄 요약 + 상세 설명 + 근거 문서):"""
    )
    | llm
    | StrOutputParser()
)

print("✅ 다중 쿼리 RetrievalQA 체인 구성 완료")
print("\n[체인 구조]")
print("""
    원본 질문
        ↓
    다중 쿼리 생성 (3개)
        ↓
    4개 쿼리로 각각 검색 (원본 + 3개)
        ↓
    검색 결과 통합 (중복 제거)
        ↓
    통합된 문서로 답변 생성
        ↓
    최종 답변 (더 포괄적)
""")


2.2 다중 쿼리 RetrievalQA 체인 구현
✅ 다중 쿼리 RetrievalQA 체인 구성 완료

[체인 구조]

    원본 질문
        ↓
    다중 쿼리 생성 (3개)
        ↓
    4개 쿼리로 각각 검색 (원본 + 3개)
        ↓
    검색 결과 통합 (중복 제거)
        ↓
    통합된 문서로 답변 생성
        ↓
    최종 답변 (더 포괄적)



In [ ]:
print("\n" + "="*80)
print("2.2 기본 RAG vs 다중 쿼리 RAG 답변 비교")
print("="*80)

# 테스트 질문
comparison_question = "Embedding 모델은 왜 중요한가요?"

print(f"질문: '{comparison_question}'\n")

print("━" * 80)
print("🔹 기본 RAG 답변 (단일 쿼리)")
print("━" * 80)
basic_answer = rag_chain.invoke(comparison_question)
print(basic_answer)

print("\n" + "━" * 80)
print("🔸 다중 쿼리 RAG 답변 (4개 쿼리)")
print("━" * 80)
multi_answer = multi_query_rag_chain.invoke({"question": comparison_question})
print(multi_answer)

print("\n" + "=" * 80)
print("💡 비교 분석")
print("=" * 80)
print("""
기본 RAG:
  • 단일 쿼리로 검색 → 4개 문서
  • 특정 관점의 정보만 수집 가능
  • 일부 관련 정보를 놓칠 수 있음

다중 쿼리 RAG:
  • 4개 쿼리로 검색 → 더 많은 문서
  • 다양한 관점: "중요성", "선택 기준", "사용 방법" 등
  • 더 포괄적이고 완전한 답변
  • 놓쳤을 수 있는 관련 문서 발견

결론:
  → 복잡하거나 다면적인 질문일수록 다중 쿼리 효과가 큼
  → 검색 시간은 증가하지만 답변 품질 향상
""")


2.2 기본 RAG vs 다중 쿼리 RAG 답변 비교
질문: 'Embedding 모델은 왜 중요한가요?'

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
🔹 기본 RAG 답변 (단일 쿼리)
━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
Embedding 모델은 정보 검색 및 데이터 처리에서 중요한 역할을 합니다. 문서에서 언급된 바와 같이, Embedding은 검색을 위한 데이터의 표현을 생성하는 과정으로, 이를 통해 정보의 효율적인 검색과 처리 성능을 향상시킬 수 있습니다. 예를 들어, OpenAI의 RAG 전략에서는 문서의 청크 크기를 실험함으로써 성능이 크게 향상되었다고 합니다. 청크 크기는 컨텍스트 윈도우에 로드되는 정보의 양을 제어하며, 이는 인덱스 구축의 핵심 단계입니다.

또한, Embedding을 통해 대량의 텍스트에서 중복된 세부 정보를 줄이고, 요약이나 작은 청크와 같은 다양한 표현을 생성하여 검색 성능을 개선할 수 있습니다. 예를 들어, 다음과 같은 코드를 사용하여 OpenAI의 Embedding 모델을 생성할 수 있습니다:

```python
embeddings = OpenAIEmbeddings(model="text-embedding-3-large")
```

이처럼 Embedding 모델은 데이터의 다양한 표현을 가능하게 하여, LLM(대형 언어 모델)과의 상호작용을 최적화하고, 정보 검색의 효율성을 높이는 데 기여합니다.

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
🔸 다중 쿼리 RAG 답변 (4개 쿼리)
━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━